
## <center> Creating model with swarming and obtain result </center>
### <center> For resampled data (with frequency 1./5) </center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io
from operator import itemgetter
import pandas as pd
from sklearn.cross_validation import train_test_split

import csv
import nupic

In [2]:
traindf = pd.read_csv('./lab_data/24chan/24chan_train_resample.csv', header = 0, skiprows = [1,2])
testdf = pd.read_csv('./lab_data/24chan/24chan_test_resample.csv', header = 0, skiprows = [1,2])

print traindf.shape, testdf.shape
print traindf.head()

(20144, 24) (20121, 24)
        field1       field2       field3      field4      field5      field6  \
0 -1698.267973 -2227.094048 -3371.315295  -97.694010  127.799416 -506.481208   
1 -3369.874326 -4316.491042 -6588.686680 -135.312760  255.609132 -993.467705   
2 -2811.257545 -3658.006301 -5531.576713 -133.670115  213.237065 -831.317236   
3 -3179.638859 -4083.630729 -6209.133002 -124.761629  242.276280 -935.320482   
4 -2901.991291 -3765.819520 -5697.672109 -133.261729  219.803744 -855.493145   

         field7        field8        field9      field10       ...        \
0  -6300.111766  -5321.451151 -29131.319062 -4542.698801       ...         
1 -12284.616892 -10340.446605 -56461.443767 -8903.983556       ...         
2 -10307.883984  -8682.424745 -47911.824102 -7490.667516       ...         
3 -11575.337707  -9736.468248 -53493.046353 -8418.065282       ...         
4 -10623.974871  -8946.833963 -49270.966940 -7730.627723       ...         

        field16      field17      fiel

In [3]:
traindf.describe()

,field1,field2,field3,field4,field5,field6,field7,field8,field9,field10,...,field16,field17,field18,field19,field20,field21,field22,field23,field24,classification
count,20144.000000,20144.000000,20144.000000,20144.000000,20144.000000,20144.000000,20144.000000,20144.000000,20144.000000,20144.000000,...,20144.000000,20144.000000,20144.000000,20144.000000,20144.000000,20144.000000,20144.000000,20144.000000,20144.000000,20144.000000
mean,-3283.819059,-3658.182920,-5900.389288,192.092280,269.692334,-940.074065,-10884.284310,-8987.883506,-49181.831690,-8079.167572,...,-31609.668233,-1561.627804,1093.810430,-118.735434,-7244.681650,-3434.810036,-6818.156027,-13077.454052,-0.017879,1.003723
std,181.924391,129.212129,80.203813,165.718331,22.290095,32.928172,141.599608,174.902010,2152.111690,113.664961,...,755.817017,276.638376,31.547862,90.151999,87.693655,265.088787,88.179483,164.186720,0.000211,0.818502
min,-4028.050150,-4316.491042,-6640.891347,-145.265758,-9.234217,-1085.404554,-12284.616892,-10340.446605,-56461.443767,-9109.152587,...,-35496.177903,-2377.788449,-45.188864,-306.322645,-8083.450903,-4353.887321,-7623.488932,-14602.891801,-0.019854,0.000000
25%,-3427.952777,-3725.429811,-5934.587649,81.236329,255.670677,-946.159828,-10907.959454,-9095.593796,-50648.596924,-8131.300679,...,-32253.390786,-1761.729326,1064.012936,-180.861573,-7257.239782,-3633.704041,-6847.803083,-13135.430509,-0.017882,0.000000
50%,-3259.633835,-3633.646659,-5908.488061,186.019319,275.411747,-930.373922,-10867.903313,-8958.279093,-50313.091800,-8081.133839,...,-31582.044518,-1522.647365,1103.944747,-109.094746,-7244.196588,-3417.111987,-6814.471170,-13062.369571,-0.017881,1.000000
75%,-3127.248307,-3555.199338,-5866.654132,352.032001,286.965925,-919.489991,-10847.760887,-8881.524786,-47923.277733,-8019.419216,...,-31046.414917,-1318.702656,1114.649251,-35.425556,-7233.327813,-3209.791653,-6785.694421,-13027.766901,-0.017881,2.000000
max,172.058130,171.152987,237.574377,467.762657,314.264459,42.207568,443.291709,392.043042,2288.714238,332.042597,...,1337.597109,133.883481,1219.769934,26.672521,286.207118,196.626208,274.543825,521.670860,0.000702,2.000000


In [4]:
#Number of fields without class label
num_fields = len(traindf.columns) - 1
print num_fields

23


In [5]:
# Exclude the column 'field14'
fields_names = ['field' + (str(i+1) if (i+1 < 14) else str(i+2)) for i in range(num_fields )] #+\
#    ['classification']
included_fields = []
for f in fields_names:
    field_dict = {'fieldName': f, 'fieldType': 'float', 'maxValue': 100000.0, 'minValue': -100000.0}
    included_fields.append(field_dict)

included_fields.append({'fieldName': 'classification', 'fieldType': 'int', 'maxValue': 2, 'minValue': 0})

In [6]:
for i in included_fields:
    print i

{'minValue': -100000.0, 'fieldName': 'field1', 'fieldType': 'float', 'maxValue': 100000.0}
{'minValue': -100000.0, 'fieldName': 'field2', 'fieldType': 'float', 'maxValue': 100000.0}
{'minValue': -100000.0, 'fieldName': 'field3', 'fieldType': 'float', 'maxValue': 100000.0}
{'minValue': -100000.0, 'fieldName': 'field4', 'fieldType': 'float', 'maxValue': 100000.0}
{'minValue': -100000.0, 'fieldName': 'field5', 'fieldType': 'float', 'maxValue': 100000.0}
{'minValue': -100000.0, 'fieldName': 'field6', 'fieldType': 'float', 'maxValue': 100000.0}
{'minValue': -100000.0, 'fieldName': 'field7', 'fieldType': 'float', 'maxValue': 100000.0}
{'minValue': -100000.0, 'fieldName': 'field8', 'fieldType': 'float', 'maxValue': 100000.0}
{'minValue': -100000.0, 'fieldName': 'field9', 'fieldType': 'float', 'maxValue': 100000.0}
{'minValue': -100000.0, 'fieldName': 'field10', 'fieldType': 'float', 'maxValue': 100000.0}
{'minValue': -100000.0, 'fieldName': 'field11', 'fieldType': 'float', 'maxValue': 100000.

In [7]:
SWARM_CONFIG = {
    "includedFields": included_fields,
    "streamDef": {
        "info": "classification_24channels",
        "version": 1,
        "Ьщвуд": [
            {
                "info": "24chan",
                "source": "file:///home/polina/Documents/hse/course_work/lab_data/24chan/24chan_test_resample.csv",
                "columns": [
                  "*"
                ]
            }
        ]
    },
    "inferenceType": "NontemporalClassification",
    "inferenceArgs": {
        "predictionSteps": [
          0
        ],
        "predictedField": u'classification'
    },
    "swarmSize": "medium"
}

In [8]:
from nupic.swarming import permutations_runner
from nupic.swarming.permutations_runner import DEFAULT_OPTIONS
import nupic.swarming

In [9]:
def swarm_over_data():
    return nupic.swarming.permutations_runner.runWithConfig(SWARM_CONFIG, {'maxWorkers': 8, 'overwrite': True})

model_params = swarm_over_data()

Generating experiment files in directory: /home/polina/Documents/hse/course_work...
Writing  313 lines...
Writing  114 lines...
done.
None
Successfully submitted new HyperSearch job, jobID=1022
##>> UPDATED WORKER STATE: 
{   u'activeSwarms': [   u'modelParams|sensorParams|encoders|field8',
                         u'modelParams|sensorParams|encoders|field9',
                         u'modelParams|sensorParams|encoders|field6',
                         u'modelParams|sensorParams|encoders|field7',
                         u'modelParams|sensorParams|encoders|field4',
                         u'modelParams|sensorParams|encoders|field5',
                         u'modelParams|sensorParams|encoders|field2',
                         u'modelParams|sensorParams|encoders|field3',
                         u'modelParams|sensorParams|encoders|field1',
                         u'modelParams|sensorParams|encoders|field18',
                         u'modelParams|sensorParams|encoders|field19',
      

In [10]:
from nupic.frameworks.opf.modelfactory import ModelFactory
model = ModelFactory.create(model_params)
model.enableInference({"predictedField": 'classification'})

In [10]:
model.save("/home/polina/Documents/hse/course_work/lab_data/24chan/model24_resample")
#model.load("/home/polina/Documents/hse/course_work/lab_data/24chan/model24_resample")

In [11]:
# Function return dataframe with 2 columns: classification value and predicted values,
# obtained with model.
# Parameters:
#  -- data_frame - pandas df
#  -- class_column - column from data_frame with class labels
#  -- model - model, obtained with nupic (class <nupic.frameworks.opf.clamodel.CLAModel>)
def predicted_values(data_frame, class_column, model):
    output = zip(np.asarray(class_column), np.zeros(len(class_column), dtype = int))
    output = pd.DataFrame(output)
    input_dict = data_frame.to_dict(orient = "index")
    for k in input_dict.keys():
        row = input_dict[k]
        result = model.run(row)
        output[1][k] = result.inferences['multiStepBestPredictions'][0]
    return output

def mean_error(init_class, predicted_class):
    return np.mean(np.array(init_class) != np.array(predicted_class))

In [12]:
test_output = predicted_values(testdf, testdf['classification'], model)
train_output =  predicted_values(traindf, traindf['classification'], model)
print test_output.head()
print train_output.head()
print np.unique(test_output[1])
print np.unique(train_output[1])

   0    1
0  1    0
1  1  127
2  1  127
3  1  127
4  1  127
   0  1
0  1  0
1  1  0
2  1  0
3  1  0
4  1  0
[  0 127 254]
[  0 127 254]


In [13]:
test_output[1] = test_output[1] // 127
train_output[1] = train_output[1] // 127
print np.unique(test_output[1])
print np.unique(train_output[1])

[0 1 2]
[0 1 2]


In [14]:
train_error = mean_error(train_output[0], train_output[1])
test_error  = mean_error(test_output[0], test_output[1])
print train_error, test_error

0.00233320095314 0.00144128025446
